# Compta Fish Academy

In [61]:
import pandas as pd
import numpy as np

df = pd.read_csv('FishAcademyCompta - transactions.csv')

def convert_series_to_date(_series) :
    return pd.to_datetime(_series, format = '%d/%m/%Y %H:%M:%S', dayfirst = True)

def convert_euros_to_float(_value) :
    return float(_value.replace('€', '').replace(',', '.').replace(' ', ''))
    
df['date'] = convert_series_to_date(df['date'])
df["montant"] = df["montant"].apply(lambda x: convert_euros_to_float(x)).astype(float)
df

,session,date,joueur,type,montant,beneficiaire,methode,notes
0,Session CG #1,2024-06-18 19:00:00,Arnaud,BUY_CHIPS,-10.00,NaN,NaN,NaN
1,Session CG #1,2024-06-18 19:00:00,Sebastien,BUY_CHIPS,-10.00,NaN,NaN,NaN
2,Session CG #1,2024-06-18 19:00:00,Eric,BUY_CHIPS,-10.00,NaN,NaN,NaN
3,Session CG #1,2024-06-18 19:00:00,Kevin,BUY_CHIPS,-9.00,NaN,NaN,NaN
4,Session CG #1,2024-06-18 19:00:00,JC,BUY_CHIPS,-10.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
112,Session CG #3,2024-07-17 00:15:00,Sebastien,TRANSFERT,-17.00,Adrien,PAYPAL,NaN
113,Session CG #3,2024-07-17 00:15:00,Sebastien,TRANSFERT,4.95,Eric,PAYPAL,NaN
114,Session CG #3,2024-07-17 00:15:00,Eric,TRANSFERT,-4.95,Sebastien,PAYPAL,NaN
115,Session CG #3,2024-07-17 00:15:00,Willy,TRANSFERT,26.50,Sebastien,PAYPAL,NaN


In [63]:
# Session CG
PARAM_CASHGAME_SESSION = 'Session CG #2'

df_session = df.loc[df.session==PARAM_CASHGAME_SESSION]

players = {}
for p in df_session.joueur.unique(): players[p] = {}

print("Calcul Gain/Perte de la session...\n")
for player in df_session.joueur.unique() :
    
    df_player_session = df_session.loc[df_session.joueur==player]
    
    total_buyin = -round(df_player_session.loc[df_player_session.type=='BUY_CHIPS','montant'].sum().item(),2)
    sold_chips = round(df_player_session.loc[df_player_session.type=='SELL_CHIPS','montant'].sum().item(),2)
    print(f"{player} est entré avec {total_buyin} EUR et est sorti avec {sold_chips} EUR")
    if total_buyin < sold_chips :
        print(f"  -> Gain de {round((sold_chips-total_buyin),2)} EUR\n")
    else :
        print(f"  -> Perte de {round((total_buyin-sold_chips),2)} EUR\n")
    players[player] = {
        'total_buyin': round(total_buyin,2),
        'sold_chips': round(sold_chips,2),
        'balance_chips': round((sold_chips - total_buyin),2)
    }

# Courses
df_courses = df_session.loc[df_session.type=='COURSES']
if df_courses.shape[0]==1 :
    players[df_courses.joueur.iloc[0]]['repas'] = round(df_courses.montant.iloc[0].item(),2)
else :
    print("Personne a fait les courses ?")
df_courses = df_session.loc[df_session.type=='REPAS']
for index, row in df_courses.iterrows():
    players[row['joueur']]['repas'] = round(row['montant'],2)


for player in df_session.joueur.unique() :
    df_transfert = df_session.loc[df_session.type=='TRANSFERT']
    total_paid = df_transfert.loc[df_session.joueur==player,'montant'].sum().item()
    total_received = df_transfert.loc[df_session.beneficiaire==player,'montant'].sum().item()
    players[player]['paid'] = round(total_paid,2)
    players[player]['received'] = round(total_received,2)
    players[player]['final_balance'] = round(players[player]['balance_chips'] 
                                             + total_paid 
                                             - total_received
                                             + players[player]['repas'] 
                                             , 2)

players

Calcul Gain/Perte de la session...

Sebastien est entré avec 25.0 EUR et est sorti avec 7.55 EUR
  -> Perte de 17.45 EUR

Arnaud est entré avec 40.0 EUR et est sorti avec 14.5 EUR
  -> Perte de 25.5 EUR

Olivier est entré avec 10.0 EUR et est sorti avec 6.15 EUR
  -> Perte de 3.85 EUR

Quentin est entré avec 10.0 EUR et est sorti avec 30.9 EUR
  -> Gain de 20.9 EUR

Adrien est entré avec 15.0 EUR et est sorti avec 17.85 EUR
  -> Gain de 2.85 EUR

Kevin est entré avec 20.0 EUR et est sorti avec 69.5 EUR
  -> Gain de 49.5 EUR

JC est entré avec 10.0 EUR et est sorti avec 0.0 EUR
  -> Perte de 10.0 EUR

Willy est entré avec 15.0 EUR et est sorti avec 0.0 EUR
  -> Perte de 15.0 EUR

Eric est entré avec 20.0 EUR et est sorti avec 18.55 EUR
  -> Perte de 1.45 EUR



{'Sebastien': {'total_buyin': 25.0,
  'sold_chips': 7.55,
  'balance_chips': -17.45,
  'repas': 49.0,
  'paid': -32.55,
  'received': 32.55,
  'final_balance': -33.55},
 'Arnaud': {'total_buyin': 40.0,
  'sold_chips': 14.5,
  'balance_chips': -25.5,
  'repas': -7.0,
  'paid': 32.5,
  'received': -32.5,
  'final_balance': 32.5},
 'Olivier': {'total_buyin': 10.0,
  'sold_chips': 6.15,
  'balance_chips': -3.85,
  'repas': 0.0,
  'paid': 4.0,
  'received': -4.0,
  'final_balance': 4.15},
 'Quentin': {'total_buyin': 10.0,
  'sold_chips': 30.9,
  'balance_chips': 20.9,
  'repas': -7.0,
  'paid': -13.9,
  'received': 13.9,
  'final_balance': -13.9},
 'Adrien': {'total_buyin': 15.0,
  'sold_chips': 17.85,
  'balance_chips': 2.85,
  'repas': -7.0,
  'paid': 5.0,
  'received': -5.0,
  'final_balance': 5.85},
 'Kevin': {'total_buyin': 20.0,
  'sold_chips': 69.5,
  'balance_chips': 49.5,
  'repas': -7.0,
  'paid': -42.5,
  'received': 42.5,
  'final_balance': -42.5},
 'JC': {'total_buyin': 10.0,
 

In [64]:
df

,session,date,joueur,type,montant,beneficiaire,methode,notes
0,Session CG #1,2024-06-18 19:00:00,Arnaud,BUY_CHIPS,-10.00,NaN,NaN,NaN
1,Session CG #1,2024-06-18 19:00:00,Sebastien,BUY_CHIPS,-10.00,NaN,NaN,NaN
2,Session CG #1,2024-06-18 19:00:00,Eric,BUY_CHIPS,-10.00,NaN,NaN,NaN
3,Session CG #1,2024-06-18 19:00:00,Kevin,BUY_CHIPS,-9.00,NaN,NaN,NaN
4,Session CG #1,2024-06-18 19:00:00,JC,BUY_CHIPS,-10.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
112,Session CG #3,2024-07-17 00:15:00,Sebastien,TRANSFERT,-17.00,Adrien,PAYPAL,NaN
113,Session CG #3,2024-07-17 00:15:00,Sebastien,TRANSFERT,4.95,Eric,PAYPAL,NaN
114,Session CG #3,2024-07-17 00:15:00,Eric,TRANSFERT,-4.95,Sebastien,PAYPAL,NaN
115,Session CG #3,2024-07-17 00:15:00,Willy,TRANSFERT,26.50,Sebastien,PAYPAL,NaN


In [65]:
df_result_session = df[['session','joueur','type','montant']].loc[(df.type=='BUY_CHIPS') | (df.type=='SELL_CHIPS')]

#df_result_session.loc[df_result_session.type=='BUY_CHIPS', 'montant'] = -df_result_session.montant
df_result_session = df_result_session.groupby(['session','joueur','type'], as_index=False).agg(
    montant=("montant","sum")
)

#table = pd.pivot_table(data=df_result_session, index=['session','joueur'])

df_result_session = pd.pivot_table(df_result_session, values='montant', index=['session','joueur'],
                       columns=['type'], aggfunc="sum").fillna(0.0)

df_result_session['Balance'] = df_result_session.SELL_CHIPS + df_result_session.BUY_CHIPS
df_result_session

type                     BUY_CHIPS  SELL_CHIPS  Balance
session       joueur                                   
Session CG #1 Arnaud         -40.0       22.30   -17.70
              Eric           -20.0       11.60    -8.40
              JC             -10.0       14.35     4.35
              Kevin          -19.0       20.45     1.45
              Olivier        -10.0        0.00   -10.00
              Sebastien      -16.0       32.65    16.65
              Willy          -10.0       23.65    13.65
Session CG #2 Adrien         -15.0       17.85     2.85
              Arnaud         -40.0       14.50   -25.50
              Eric           -20.0       18.55    -1.45
              JC             -10.0        0.00   -10.00
              Kevin          -20.0       69.50    49.50
              Olivier        -10.0        6.15    -3.85
              Quentin        -10.0       30.90    20.90
              Sebastien      -25.0        7.55   -17.45
              Willy          -15.0        0.00   -15.00
Session CG #3 Adrien         -25.0       15.40    -9.60
              Arnaud         -20.0       45.60    25.60
              Eric           -20.0       31.95    11.95
              Kevin          -80.0       20.90   -59.10
              Olivier        -10.0       19.85     9.85
              Sebastien      -10.0       50.80    40.80
              Willy          -30.0       10.50   -19.50

,session,date,joueur,type,montant,beneficiaire,methode,notes
0,Session CG #1,2024-06-18 19:00:00,Arnaud,BUY_CHIPS,10.00,NaN,NaN,NaN
1,Session CG #1,2024-06-18 19:00:00,Sebastien,BUY_CHIPS,10.00,NaN,NaN,NaN
2,Session CG #1,2024-06-18 19:00:00,Eric,BUY_CHIPS,10.00,NaN,NaN,NaN
3,Session CG #1,2024-06-18 19:00:00,Kevin,BUY_CHIPS,9.00,NaN,NaN,NaN
4,Session CG #1,2024-06-18 19:00:00,JC,BUY_CHIPS,10.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
91,Session CG #3,2024-07-17 00:15:00,Willy,SELL_CHIPS,10.50,NaN,NaN,NaN
92,Session CG #3,2024-07-17 00:15:00,Kevin,TRANSFERT,56.10,Sebastien,PAYPAL,10€ de perte reportée
93,Session CG #3,2024-07-17 00:15:00,Adrien,TRANSFERT,17.00,Sebastien,PAYPAL,NaN
94,Session CG #3,2024-07-17 00:15:00,Sebastien,TRANSFERT,4.95,Eric,PAYPAL,NaN


In [66]:
for joueur in df.joueur.unique() :
    
    df_joueur = df.loc[df.joueur==joueur]
    
    total_buyin = -round(df_joueur[df_joueur.type=='BUY_CHIPS']['montant'].sum().item(),2)
    total_returned_chips = round(df_joueur[df_joueur.type=='SELL_CHIPS']['montant'].sum().item(),2)
    total_repas = -round(df_joueur[df_joueur.type=='REPAS']['montant'].sum().item(),2)
    total_courses = round(df_joueur[df_joueur.type=='COURSES']['montant'].sum().item(),2)
    total_courses_remboursees = round(df[(df.type=='REPAS') & (df.beneficiaire==joueur)]['montant'].sum().item(),2)
    #total_transfert_envoye = round(df_joueur[df_joueur.type=='TRANSFERT']['montant'].sum().item(),2)
    #total_transfert_recu = round(df[(df.type=='TRANSFERT') & (df.beneficiaire==joueur)]['montant'].sum().item(),2)
    balance_transfert = round(df_joueur[df_joueur.type=='TRANSFERT']['montant'].sum().item(),2)
    balance = total_returned_chips - total_buyin - total_repas + total_courses + balance_transfert

    print(f"\nBilan pour {joueur} : {round(balance,2)} €")
    print(f"  - Total achat jetons   : {total_buyin} €")
    print(f"  - Total revente jetons : {total_returned_chips} €")
    print(f"  - Balance jetons       : {round(total_returned_chips-total_buyin,2)} €")
    if total_repas>0 :
        print(f"  - Participations repas : {total_repas} €")
    if total_courses>0 :
        print(f"  - Courses payées       : {total_courses} €")
    if total_courses_remboursees>0 :
        print(f"  - Courses remboursées  : {total_courses_remboursees} €")
    print(f"  - Balance transfert    : {balance_transfert} € ({round((balance_transfert-total_repas),2)} € hors paiement repas)")
    
    


Bilan pour Arnaud : 51.2 €
  - Total achat jetons   : 100.0 €
  - Total revente jetons : 82.4 €
  - Balance jetons       : -17.6 €
  - Balance transfert    : 54.8 € (68.8 € hors paiement repas)

Bilan pour Sebastien : -17.45 €
  - Total achat jetons   : 51.0 €
  - Total revente jetons : 91.0 €
  - Balance jetons       : 40.0 €
  - Courses payées       : 91.0 €
  - Balance transfert    : -148.45 € (-148.45 € hors paiement repas)

Bilan pour Eric : 28.0 €
  - Total achat jetons   : 60.0 €
  - Total revente jetons : 62.1 €
  - Balance jetons       : 2.1 €
  - Balance transfert    : 11.9 € (25.9 € hors paiement repas)

Bilan pour Kevin : 18.0 €
  - Total achat jetons   : 119.0 €
  - Total revente jetons : 110.85 €
  - Balance jetons       : -8.15 €
  - Balance transfert    : 12.15 € (26.15 € hors paiement repas)

Bilan pour JC : 14.0 €
  - Total achat jetons   : 20.0 €
  - Total revente jetons : 14.35 €
  - Balance jetons       : -5.65 €
  - Balance transfert    : 12.65 € (19.65 € hors pa

In [51]:
round(df.loc[(df.session=='Session CG #3') & ((df.type=='BUY_CHIPS') | (df.type=='SELL_CHIPS'))]['montant'].sum(),2)

np.float64(-0.0)

In [59]:
df = pd.read_csv('next_game_registration.csv',
	usecols = ["player", "iscoming"],
	dtype = {
    	'player': 'string',
    	'iscoming': 'int'
	})
df

,player,iscoming
0,Sebastien,1
1,Quentin,0
2,JC,0
3,Adrien,0
4,Eric,0
5,Willy,0
6,Kevin,0
7,Arnaud,0


In [71]:
df.montant.sum().item()

0.0